In [2]:
import os
from dotenv import load_dotenv, find_dotenv 
_ = load_dotenv(find_dotenv()) 
openai_api_key = os.environ["OPENAI_API_KEY"]

In [4]:
from langchain_community.vectorstores import FAISS 
from langchain_openai import OpenAIEmbeddings 
from langchain_text_splitters import CharacterTextSplitter 
from langchain_community.document_loaders import TextLoader

In [22]:
loader = TextLoader("D:\Code\Langchain\data\state_of_the_union.txt", encoding="utf-8")
loaded_document = loader.load() 

<>:1: SyntaxWarning: invalid escape sequence '\C'
<>:1: SyntaxWarning: invalid escape sequence '\C'
C:\Users\ADMIN\AppData\Local\Temp\ipykernel_1944\926902079.py:1: SyntaxWarning: invalid escape sequence '\C'
  loader = TextLoader("D:\Code\Langchain\data\state_of_the_union.txt", encoding="utf-8")


In [24]:
text_splitter = CharacterTextSplitter(chunk_size = 1000, chunk_overlap = 0) 
chunks_of_text = text_splitter.split_documents(loaded_document)

In [26]:
len(chunks_of_text)

42

In [27]:
embeddings = OpenAIEmbeddings() 

vector_db = FAISS.from_documents(chunks_of_text, embeddings)

In [28]:
vector_db

In [32]:
retriever = vector_db.as_retriever() 

In [33]:
response = retriever.invoke("What did he say about ketanji brown jackson ?")

In [34]:
response

[Document(metadata={'source': 'D:\\Code\\Langchain\\data\\state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.'),
 Document(metadata={'source': 'D:\\Code\\Langchain\\data\\state_of_the_union.txt'}, page_content='One was

In [35]:
retriever = vector_db.as_retriever(search_kwargs = {"k": 1})

In [36]:
response = retriever.invoke("What did he say about ketanji brown jackson ?")

In [37]:
response

[Document(metadata={'source': 'D:\\Code\\Langchain\\data\\state_of_the_union.txt'}, page_content='Tonight. I call on the Senate to: Pass the Freedom to Vote Act. Pass the John Lewis Voting Rights Act. And while you’re at it, pass the Disclose Act so Americans can know who is funding our elections. \n\nTonight, I’d like to honor someone who has dedicated his life to serve this country: Justice Stephen Breyer—an Army veteran, Constitutional scholar, and retiring Justice of the United States Supreme Court. Justice Breyer, thank you for your service. \n\nOne of the most serious constitutional responsibilities a President has is nominating someone to serve on the United States Supreme Court. \n\nAnd I did that 4 days ago, when I nominated Circuit Court of Appeals Judge Ketanji Brown Jackson. One of our nation’s top legal minds, who will continue Justice Breyer’s legacy of excellence.')]

#### **LCEL (Langchain Expression Language)**

In [38]:
from langchain_openai import ChatOpenAI 
from langchain_core.prompts import ChatPromptTemplate 
from langchain_core.output_parsers import StrOutputParser 
from langchain_core.runnables import RunnablePassthrough

In [39]:
template = """Answer the question based only on the following context:

{context} 

Question: {question}
"""

prompt = ChatPromptTemplate.from_template(template) 

model = ChatOpenAI()

In [42]:
def format_docs(docs):
    return "\n\n".join([d.page_content for d in docs])  

chain = (
    {'context': retriever | format_docs, "question" : RunnablePassthrough()} 
    | prompt 
    | model 
    | StrOutputParser()
)

In [43]:
response = chain.invoke("What did he say about ketanji brown jackson ?")

In [44]:
response

"He said that Ketanji Brown Jackson is one of our nation's top legal minds who will continue Justice Breyer's legacy of excellence."

In [45]:
type(response)

str